In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [4]:
%%time
df = pd.read_excel('card transactions.xlsx')

CPU times: user 10.3 s, sys: 50.9 ms, total: 10.3 s
Wall time: 10.3 s


# 1. Drop Outliners and Keep Only P

In [5]:
df.columns

Index(['Recnum', 'Cardnum', 'Date', 'Merchnum', 'Merch description',
       'Merch state', 'Merch zip', 'Transtype', 'Amount', 'Fraud'],
      dtype='object')

In [6]:
#Drop one large transaction
#df['Amount'].max()
#df[df['Amount']== 3102045.53]
df = df.drop(52715)

In [7]:
#Keep only type P
df = df[df['Transtype']=='P']

# 2.Filling in missing values

In [8]:
#filling in merch zip by merch state and merchnum

In [9]:
zip_ref_1 = df.groupby(['Merch state','Merchnum'])['Merch zip'].apply(lambda x: x.mode()).reset_index()
zip_ref_1 = zip_ref_1[zip_ref_1['level_2'] == 0].iloc[:,[0,1,3]]
zip_ref_1 = zip_ref_1.rename(columns = {'Merch zip':'zip_ref_1'})

In [10]:
df = df.merge(zip_ref_1,on = ['Merch state','Merchnum'], how = 'left')

In [11]:
zip_ref_2 = df.groupby('Merch state')['Merch zip'].apply(lambda x: x.mode()).reset_index()
zip_ref_2 = zip_ref_2[zip_ref_2['level_1'] == 0].iloc[:,[0,2]]
zip_ref_2 = zip_ref_2.rename(columns = {'Merch zip':'zip_ref_2'})

In [12]:
df = df.merge(zip_ref_2,on = 'Merch state', how = 'left')

In [13]:
zip_ref_3 = df.groupby('Merchnum')['Merch zip'].apply(lambda x: x.mode()).reset_index()
zip_ref_3 = zip_ref_3[zip_ref_3['level_1'] == 0].iloc[:,[0,2]]
zip_ref_3 = zip_ref_3.rename(columns = {'Merch zip':'zip_ref_3'})

In [14]:
df = df.merge(zip_ref_3,on = 'Merchnum', how = 'left')

In [15]:
df['Merch zip'] = df['Merch zip'].fillna(df['zip_ref_1']).fillna(df['zip_ref_2']).fillna(df['zip_ref_3'])
sum(pd.isnull(df['Merch zip']))

1096

In [16]:
#drop if the value is still na
df = df.dropna(subset=['Merch zip'])
sum(pd.isnull(df['Merch zip']))

0

In [17]:
#filling in merch state by merch zip and merchnum
state_ref_1 = df.groupby(['Merch zip','Merchnum'])['Merch state'].apply(lambda x: x.mode()).reset_index()
state_ref_1 = state_ref_1[state_ref_1['level_2'] == 0].iloc[:,[0,1,3]]
state_ref_1 = state_ref_1.rename(columns = {'Merch state':'state_ref_1'})

In [18]:
df = df.merge(state_ref_1,on = ['Merch zip','Merchnum'], how = 'left')

In [19]:
state_ref_2 = df.groupby('Merch zip')['Merch state'].apply(lambda x: x.mode()).reset_index()
state_ref_2 = state_ref_2[state_ref_2['level_1'] == 0].iloc[:,[0,2]]
state_ref_2 = state_ref_2.rename(columns = {'Merch state':'state_ref_2'})

In [20]:
df = df.merge(state_ref_2,on = 'Merch zip', how = 'left')

In [21]:
state_ref_3 = df.groupby('Merchnum')['Merch state'].apply(lambda x: x.mode()).reset_index()
state_ref_3 = state_ref_3[state_ref_3['level_1'] == 0].iloc[:,[0,2]]
state_ref_3 = state_ref_3.rename(columns = {'Merch state':'state_ref_3'})

In [22]:
df = df.merge(state_ref_3,on = 'Merchnum', how = 'left')

In [23]:
df['Merch state'] = df['Merch state'].fillna(df['state_ref_1']).fillna(df['state_ref_2']).fillna(df['state_ref_3'])
sum(pd.isnull(df['Merch state']))

67

In [24]:
#drop if the value is still na
df = df.dropna(subset=['Merch state'])
sum(pd.isnull(df['Merch state']))

0

In [25]:
#filling in merchnum by merch zip and merch state
num_ref_1 = df.groupby(['Merch zip','Merch state'])['Merchnum'].apply(lambda x: x.mode()).reset_index()
num_ref_1 = num_ref_1[num_ref_1['level_2'] == 0].iloc[:,[0,1,3]]
num_ref_1 = num_ref_1.rename(columns = {'Merchnum':'num_ref_1'})

In [26]:
df = df.merge(num_ref_1,on = ['Merch zip','Merch state'], how = 'left')

In [27]:
num_ref_2 = df.groupby('Merch zip')['Merchnum'].apply(lambda x: x.mode()).reset_index()
num_ref_2 = num_ref_2[num_ref_2['level_1'] == 0].iloc[:,[0,2]]
num_ref_2 = num_ref_2.rename(columns = {'Merchnum':'num_ref_2'})

In [28]:
df = df.merge(num_ref_2,on = 'Merch zip', how = 'left')

In [29]:
num_ref_3 = df.groupby('Merch state')['Merchnum'].apply(lambda x: x.mode()).reset_index()
num_ref_3 = num_ref_3[num_ref_3['level_1'] == 0].iloc[:,[0,2]]
num_ref_3 = num_ref_3.rename(columns = {'Merchnum':'num_ref_3'})

In [30]:
df = df.merge(num_ref_3,on = 'Merch state', how = 'left')

In [31]:
df['Merchnum'] = df['Merchnum'].fillna(df['num_ref_1']).fillna(df['num_ref_2']).fillna(df['num_ref_3'])
sum(pd.isnull(df['Merchnum']))

0

In [32]:
# drop all reference columns
df = df.drop(['state_ref_1', 'state_ref_2', 'state_ref_3','zip_ref_1', 'zip_ref_2', 'zip_ref_3','num_ref_1', 'num_ref_2', 'num_ref_3'], axis=1)

In [33]:
df.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


# 3. Variable Creation

## 3.1. Amount and frequency variables

In [31]:
#days = [1,3,7,10,14,30]
aggs = {'Amount':['mean', 'median', 'max', 'sum', 'count']}

In [32]:
df.set_index('Date', inplace = True)

In [33]:
%%time
card = []
days = [1,3,7,14,30, 60]
for day in days:
    a = df.groupby(by = ['Cardnum']).Amount.rolling(str(day)+'d').agg(aggs).Amount.reset_index()
    a['order_card'] = a.groupby(['Cardnum', 'Date']).cumcount() + 1
    card.append(a)

C:\Users\DELL\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\window.py:1561: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(Rolling, self).aggregate(arg, *args, **kwargs)


Exception: Column(s) Amount already selected

In [7]:
%%time

merch = []
for day in days:
    a = df[['Merchnum', 'Amount']].groupby(by = 'Merchnum')['Amount'].rolling(str(day)+'d').agg(aggs).Amount.reset_index()
    a['order_merch'] = a.groupby(['Date', 'Merchnum']).cumcount() + 1
    merch.append(a)

C:\Users\vandi\Anaconda3\lib\site-packages\pandas\core\window.py:1685: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(Rolling, self).aggregate(arg, *args, **kwargs)


KeyboardInterrupt: 

In [ ]:
%%time
state = []
for day in days:
    a = df[['Cardnum', 'Merch state', 'Amount']].groupby(by = ['Cardnum','Merch state']).Amount.rolling(str(day)+'d')\
    .agg(aggs).Amount.reset_index()
    a['order_state'] = a.groupby(['Date', 'Merch state', 'Cardnum']).cumcount() + 1
    state.append(a)

In [ ]:
%%time
zip = []
for day in days:
    a = df.groupby(by = ['Cardnum' ,'Merch zip']).Amount.rolling(str(day)+'d').agg(aggs).Amount.reset_index()
    a['order_zip'] = a.grouby(['Date', 'Merch zip', 'Cardnum']).cumcount() + 1
    zip.append(a)

In [252]:
%%time
cardmerch = []
for day in days:
    a = df.groupby(by = ['Cardnum','Merchnum']).Amount.rolling(str(day)+'d').agg(aggs).Amount.reset_index()
    a['order_mc'].groupby(['Date', 'Merchnum', 'Cardnum'])
    cardmerch.append(a)

KeyboardInterrupt: 

In [14]:
df['order_merch'] = df.groupby(by = ['Date', 'Merchnum']).cumcount() + 1
df['order_card'] = df.groupby(by = ['Date', 'Cardnum']).cumcount() + 1
df['order_mc'] = df.groupby(by = ['Date', 'Cardnum', 'Merchnum']).cumcount() + 1
df['order_zip'] = df.groupby(by = ['Date', 'Cardnum', 'Merch zip']).cumcount() + 1
df['order_state'] = df.groupby(by = ['Date', 'Cardnum', 'Merch state']).cumcount() + 1

In [8]:
temp = card[0]
i = 0
for c in card[1:]:
    y = days[i]
    x = days[1+i]
    i+=1
    temp = pd.merge(left = temp, right = c, how = 'inner', on = ['Date', 'order_card', 'Cardnum'], suffixes = \
                    ('card_day'+str(y), 'card_day'+str(x)))
temp.head()

,Cardnum,Date,meancard_day1,mediancard_day1,maxcard_day1,sumcard_day1,countcard_day1,order_card,meancard_day3,mediancard_day3,...,meancard_day30,mediancard_day30,maxcard_day30,sumcard_day30,countcard_day30,meancard_day60,mediancard_day60,maxcard_day60,sumcard_day60,countcard_day60
0,5142110002,2010-10-12,150.00,150.00,150.0,150.0,1.0,1,150.00,150.00,...,150.00,150.00,150.0,150.0,1.0,150.00,150.00,150.0,150.0,1.0
1,5142110081,2010-03-08,495.90,495.90,495.9,495.9,1.0,1,495.90,495.90,...,495.90,495.90,495.9,495.9,1.0,495.90,495.90,495.9,495.9,1.0
2,5142110081,2010-03-08,566.05,566.05,636.2,1132.1,2.0,2,566.05,566.05,...,566.05,566.05,636.2,1132.1,2.0,566.05,566.05,636.2,1132.1,2.0
3,5142110081,2010-11-26,636.20,636.20,636.2,636.2,1.0,1,636.20,636.20,...,636.20,636.20,636.2,636.2,1.0,636.20,636.20,636.2,636.2,1.0
4,5142110081,2010-12-27,495.90,495.90,495.9,495.9,1.0,1,495.90,495.90,...,495.90,495.90,495.9,495.9,1.0,566.05,566.05,636.2,1132.1,2.0


In [9]:
#

In [10]:
merchs = merch[0]
i = 0
for c in merch[1:]:
    y = days[i]
    x = days[1+i]
    i+=1
    merchs = pd.merge(left = merchs, right = c, how = 'inner', on = ['Date', 'order_merch', 'Merchnum'], suffixes = \
                    ('merch_day'+str(y), 'merch_day'+str(x)))
merchs.head()

NameError: name 'merch' is not defined

In [ ]:
states = state[0]
i = 0
for c in state[1:]:
    x = days[1+i]
    i+=1
    states = pd.merge(left = states, right = c, how = 'inner', on = ['Date', 'order_state', 'Cardnum', 'Merch state'], suffixes = \
                    ('state_day'+str(x), 'state_day'+str(x)))
states.head()

In [ ]:
zips = zip[0]
i = 0
for c in zip[1:]:
    y = days[i]
    x = days[1+i]
    i+=1
    zips = pd.merge(left = zips, right = c, how = 'inner', on = ['Date', 'order', 'Cardnum', 'Merch zip'], suffixes = \
                    ('zip_day'+str(y), 'zip_day'+str(x)))
zips.head()

In [ ]:
mc = cardmerch[0]
i = 0
for c in cardmerch[1:]:
    y = days[i]
    x = days[1+i]
    i+=1
    mc = pd.merge(left = mc, right = c, how = 'inner', on = ['Date', 'order', 'Cardnum', 'Merchnum'], suffixes = \
                    ('mc_day'+str(y), 'mc_day'+str(x)))
mc.head()

# merging tables

In [11]:
df = df.merge(temp, how = 'left', on = ['Date', 'Cardnum', 'order_card'], suffixes = ('', '_card'))

In [268]:
df = df.merge(merchs, how = 'left', on = ['Date', 'Merchnum', 'order_merch'], suffixes = ('', '_merch'))

In [272]:
df = df.merge(mc, how = 'left', on = ['Date', 'Cardnum', 'Merchnum', 'order_mc'], suffixes = ('', '_mc'))

NameError: name 'mc' is not defined

In [273]:
df = df.merge(zips, how = 'left', on = ['Date', 'Cardnum', 'Merch zip', 'order_zip'], suffixes = ('', '_zip'))

NameError: name 'zips' is not defined

In [274]:
df = df.merge(states, how = 'left', on = ['Date', 'Cardnum', 'Merch state', 'order_state'], suffixes = ('', '_state'))

In [48]:
#DAY 0

In [15]:
z = df[df.columns[:10]].groupby(['Cardnum', 'Date']).Amount.agg(aggs).Amount.reset_index()
z['order_card'] = z.groupby(['Cardnum', 'Date']).cumcount()+1

C:\Users\vandi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [16]:
z2 = df[df.columns[:10]].groupby(['Merchnum', 'Date']).Amount.agg(aggs).Amount.reset_index()
z2['order_merch'] = z2.groupby(['Merchnum', 'Date']).cumcount()+1

C:\Users\vandi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [17]:
z3 = df[df.columns[:10]].groupby(['Merchnum','Cardnum', 'Date']).Amount.agg(aggs).Amount.reset_index()
z3['order_mc'] = z3.groupby(['Merchnum','Cardnum', 'Date']).cumcount()+1

C:\Users\vandi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [18]:
z4 = df[df.columns[:10]].groupby(['Cardnum','Merch state', 'Date']).Amount.agg(aggs).Amount.reset_index()
z4['order_state'] = z4.groupby(['Cardnum','Merch state', 'Date']).cumcount()+1

C:\Users\vandi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [19]:
z5 = df[df.columns[:10]].groupby(['Cardnum','Merch zip', 'Date']).Amount.agg(aggs).Amount.reset_index()
z5['order_zip'] = z5.groupby(['Cardnum','Merch zip', 'Date']).cumcount()+1

C:\Users\vandi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [20]:
df = df.merge(z, how = 'left', on = ['Cardnum', 'Date', 'order_card'], suffixes = ('','_card_day0'))


In [21]:
df = df.merge(z2, how = 'left', on = ['Merchnum', 'Date', 'order_merch'], suffixes = ('','_merch_day0'))

In [22]:
df = df.merge(z3, how = 'left', on = ['Merchnum', 'Date','Cardnum', 'order_mc'], suffixes = ('','_mc_day0'))

In [23]:
df = df.merge(z4, how = 'left', on = ['Merch state', 'Date','Cardnum', 'order_state'], suffixes = ('','_state_day0'))

In [24]:
df = df.merge(z5, how = 'left', on = ['Merch zip', 'Date','Cardnum', 'order_zip'], suffixes = ('','_zip_day0'))

In [25]:
c = df.columns[15:]

In [26]:
j = 0
for col in c:
    j = j%5
    if j==4:
        j+=1
        continue
    j+=1
    df['Amount/'+str(col)] = df['Amount']/df[col]

In [27]:
c= df.columns[15:]

In [28]:
len(z4.columns)


9

In [29]:
a =  df.groupby('Cardnum').Amount.rolling('1d').count()
#a['rank'] = a.reset_index().groupby(['Cardnum', 'Date']).cumcount()
a.head(10).reset_index()

ValueError: window must be an integer

# Days since variable

In [30]:
def second_last(x):
    a = sorted(x, reverse = True)
    if len(a) ==1:
        return 0
    else:
        return a[1]

In [31]:
a = df.set_index('Date').groupby([ 'Cardnum']).Recnum.rolling('365d').apply(lambda x: second_last(x)).reset_index()
a['order_card'] = a.groupby(['Cardnum']).cumcount()+1
a= a.merge(df[['Recnum', 'Date']], how = 'inner', suffixes = ('','_previous_card'), on = 'Recnum')

C:\Users\vandi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


In [32]:
b = df.set_index('Date').groupby([ 'Merchnum']).Recnum.rolling('365d').apply(lambda x: second_last(x)).reset_index()
b['order_merch'] = b.groupby(['Merchnum']).cumcount()+1
b= b.merge(df[['Recnum', 'Date']], how = 'inner', suffixes = ('','_previous_merch'), on = 'Recnum')

C:\Users\vandi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


KeyboardInterrupt: 

In [ ]:
c = df.set_index('Date').groupby([ 'Cardnum', 'Merch zip']).Recnum.\
rolling('365d').apply(lambda x: second_last(x)).reset_index()

c['order_zip'] = c.groupby(['Cardnum', 'Merch zip']).cumcount()+1
c = c.merge(df[['Recnum', 'Date']], how = 'inner', suffixes = ('','_previous_zip'), on = 'Recnum')

In [214]:
d = df.set_index('Date').groupby([ 'Cardnum', 'Merch state']).Recnum.\
rolling('365d').apply(lambda x: second_last(x)).reset_index()

d['order_state'] = d.groupby(['Cardnum', 'Merch state']).cumcount()+1

d = d.merge(df[['Recnum', 'Date']], how = 'inner', suffixes = ('','_previous_state'), on = 'Recnum')

C:\Users\vandi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  


In [215]:
e = df.set_index('Date').groupby([ 'Cardnum', 'Merchnum']).Recnum.\
rolling('365d').apply(lambda x: second_last(x)).reset_index()

e['order_mc'] = e.groupby(['Cardnum', 'Merchnum']).cumcount()+1

e = e.merge(df[['Recnum', 'Date']], how = 'inner', suffixes = ('','_previous_mc'), on = 'Recnum')

C:\Users\vandi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  


In [216]:
del a['Recnum']
del b['Recnum']
del e['Recnum']
del d['Recnum']
del c['Recnum']


In [217]:
#df = 

df = df.merge(e, on = ['Cardnum', 'Merchnum', 'Date', 'order_mc'],how = 'left', suffixes = ('',''))

df = df.merge(a, on = ['Cardnum', 'Date', 'order_card'], suffixes = ('',''), how = 'left')

df = df.merge(c, on = ['Cardnum', 'Merch zip', 'Date', 'order_zip'], suffixes = ('',''), how = 'left')

df = df.merge(b, on = [ 'Merchnum', 'Date', 'order_merch'], suffixes = ('',''), how = 'left')

In [218]:
df = df.merge(d, on = ['Cardnum', 'Merch state', 'Date', 'order_state'], suffixes = ('',''), how = 'left')

In [243]:
#start with date operations

In [226]:
df['Date']= df['Date'].apply(lambda x: x.date())


In [236]:
x = df['Date']



364

In [244]:
(df[['Date']] - df[['Date']] )

# KS test

In [51]:
from scipy.stats import ks_2samp

In [52]:
d = {}
b = df[df['Fraud']==1]
a = df[df['Fraud']==0]
for i in df.columns[15:]:
    x = a[i]
    y = b[i] 

    d[i] = ks_2samp(x, y)[0]

In [53]:
l = []
[l.append([key, float(val)]) for key, val in d.items()]

ks = pd.DataFrame(l, columns = ['Column', 'KS Score'])

In [54]:
ks = ks.sort_values(by = 'KS Score', ascending=False)

In [55]:
#number of ks above 50%
sum(ks['KS Score']>0.5)

0

In [56]:
ks.tail()

,Column,KS Score
9,count_merch_day0,0.187478
29,Amount/mean_merch_day0,0.181263
30,Amount/median_merch_day0,0.178918
32,Amount/sum_merch_day0,0.163698
31,Amount/max_merch_day0,0.163698


In [57]:
ks.head()

,Column,KS Score
18,sum_state_day0,0.481805
23,sum_zip_day0,0.481415
13,sum_mc_day0,0.477926
17,max_state_day0,0.476706
22,max_zip_day0,0.475236


In [34]:
df.columns

Index(['Recnum', 'Date', 'Cardnum', 'Merchnum', 'Merch description',
       'Merch state', 'Merch zip', 'Transtype', 'Amount', 'Fraud',
       'order_merch', 'order_card', 'order_mc', 'order_zip', 'order_state',
       'mean', 'median', 'max', 'sum', 'count', 'mean_merch_day0',
       'median_merch_day0', 'max_merch_day0', 'sum_merch_day0',
       'count_merch_day0', 'mean_mc_day0', 'median_mc_day0', 'max_mc_day0',
       'sum_mc_day0', 'count_mc_day0', 'mean_state_day0', 'median_state_day0',
       'max_state_day0', 'sum_state_day0', 'count_state_day0', 'mean_zip_day0',
       'median_zip_day0', 'max_zip_day0', 'sum_zip_day0', 'count_zip_day0',
       'Amount/mean', 'Amount/median', 'Amount/max', 'Amount/sum',
       'Amount/mean_merch_day0', 'Amount/median_merch_day0',
       'Amount/max_merch_day0', 'Amount/sum_merch_day0', 'Amount/mean_mc_day0',
       'Amount/median_mc_day0', 'Amount/max_mc_day0', 'Amount/sum_mc_day0',
       'Amount/mean_state_day0', 'Amount/median_state_day0',
 